In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, BertModel, AutoTokenizer, AutoModel
from tqdm import tqdm

from dataset import NamesDataset

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

train_dict_path = 'train_dict.json'
val_dict_path = 'val_dict.json'

trainset = NamesDataset(train_dict_path, tokenizer=tokenizer, max_length=32)
valset = NamesDataset(val_dict_path, tokenizer=tokenizer, max_length=32)


trainset_labels = set(trainset.labels)
other_class_idx = 'other'
for idx, label in tqdm(enumerate(valset.labels)):
    if label not in trainset_labels:
        valset.labels[idx] = other_class_idx
valset.labels2target = trainset.labels2target
    

print('Train size:', len(trainset))
print('Val size:', len(valset))

/mnt/disk2/ndubrovnyi/words_similarity/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
3878it [00:00, 3178099.04it/s]

Train size: 14342
Val size: 3878


In [3]:
import torch

from model import NamesRecognition, ArcFace, AdaCos
from utils import get_faiss_index
from torch.utils.data import DataLoader


device = 'cuda:4'

margin = 0.5
scale = 15
num_classes = len(set(trainset.labels))
feature_extractor_name = 'bert-base-uncased'
embedding_size = 512

other_label = trainset.labels2target['other']

num_epochs = 30

train_loader = DataLoader(trainset, batch_size=64, shuffle=False)
val_loader = DataLoader(valset, batch_size=64, shuffle=False)

model = torch.load('SaveModels/BertArcFace_v4_23.pth', map_location=device)

model.to(device)

faiss_index = get_faiss_index(model, train_loader, device, embedding_size)

100%|█████████████████████████████████████████| 225/225 [00:09<00:00, 23.49it/s]


In [20]:
from utils import get_similarity_names
import random

name = random.choice(valset.names)

sim_names_cnt = 5
thresh = 0.5

print('Target Name:', name)
result = get_similarity_names(name, model=model, index=faiss_index, sim_names_cnt=sim_names_cnt, tokenizer=tokenizer,
                           device=device, emb_size=embedding_size, thresh=thresh)

if len(result) > 0:
    print('Similarity names:')
    for res in result:
        print(f'{trainset.get_name(res[0])}: Conf={round(res[1], 2)}')
    
else:
    print('No similarity names')

Target Name: Dongguan Sanxie Polymer Products Co., Ltd.


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 34.54it/s]

Similarity names:
Dongguan Lijin Plastic Products Co., Ltd.: Conf=0.6700000166893005
Dongguan Jinggan Plastic Products Co., Ltd.: Conf=0.6399999856948853
Dongguan Deyuan Plastic Production Co., Ltd.: Conf=0.5299999713897705
Dongguan Yi Fa Plastic Products Co., Ltd.: Conf=0.5199999809265137
Dongguan Yu Po Plastic Products Co., Ltd.: Conf=0.5099999904632568
